**Step 1: Import the libraries**

In [1]:
import pandas as pd
from datetime import datetime

**Step 2: Load dataset with Explicit Tab Separator**

In [3]:
df = pd.read_csv("/content/cleaned_marketing_campaign.csv", sep=",")

**Step 3: Normalize Column Names**

In [4]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

**Step 4: Initial Diagnostics**

In [5]:
initial_shape = df.shape
missing_before = df.isnull().sum()
duplicates_before = df.duplicated().sum()

**Step 5: Convert dt_customer to datetime (day-first)**

In [6]:
df['dt_customer'] = pd.to_datetime(df['dt_customer'], dayfirst=True, errors='coerce')

**Step 6: Compute age (prefer dt_customer year; fallback to 2025)**

In [8]:
df['year_birth'] = pd.to_numeric(df['year_birth'], errors='coerce')
def compute_age(row):
    if pd.notnull(row['dt_customer']) and pd.notnull(row['year_birth']):
        return row['dt_customer'].year - row['year_birth']
    if pd.notnull(row['year_birth']):
        return 2025 - row['year_birth']
    return pd.NA
df['age'] = df.apply(compute_age, axis=1).astype('Int64')

**Step 7: Standardize Education**

In [9]:
edu_map = {
    'basic': 'undergraduate',
    '2n cycle': 'undergraduate',
    'basic ': 'undergraduate',
    'graduation': 'graduate',
    'master': 'postgraduate',
    'phd': 'postgraduate'
}
df['education'] = df['education'].astype(str).str.strip().str.lower().replace(edu_map)

**Step 7: Standardize Marital Status**

In [10]:
marital_map = {
    'married': 'partner',
    'together': 'partner',
    'divorced': 'single',
    'single': 'single',
    'widow': 'single',
    'alone': 'single',
    'absurd': 'other',
    'yolo': 'other'
}
df['marital_status'] = df['marital_status'].astype(str).str.strip().str.lower().replace(marital_map)

**Step 7: Combine Children**

In [11]:
df['children'] = df[['kidhome', 'teenhome']].sum(axis=1)

**Step 8: Total spent across product categories**

In [12]:
spend_cols = [
    'mnt_wines', 'mnt_fruits', 'mnt_meat_products',
    'mnt_fish_products', 'mnt_sweet_products', 'mnt_gold_prods'
]
existing_spend_cols = [c for c in spend_cols if c in df.columns]
df['total_spent'] = df[existing_spend_cols].sum(axis=1)

**Step 9: Derive family feature**

In [13]:
df['is_partner'] = df['marital_status'] == 'partner'
df['family_size'] = df['is_partner'].astype(int) + df['children']

**Step 10: Income imputation**

In [14]:
df['income'] = pd.to_numeric(df['income'], errors='coerce')
income_missing_before = df['income'].isnull().sum()
income_median = df['income'].median()
df['income'] = df['income'].fillna(income_median)

**Step 11: Remove the duplicates**

In [15]:
df = df.drop_duplicates()

**Step 12: Ensure spend columns are integers**

In [16]:
for col in existing_spend_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

**Step 11: Finalize the diagnostics**

In [17]:
final_shape = df.shape
missing_after = df.isnull().sum()
duplicates_after = df.duplicated().sum()

**Step 12: Save the cleaned dataset**

In [18]:
df.to_csv("cleaned_marketing_campaign_full_cleaned.csv", index=False)

**Step 13: Print the Summary**

In [19]:
print("Initial shape:", initial_shape)
print("Final shape:", final_shape)
print("Duplicates before:", duplicates_before)
print("Duplicates after:", duplicates_after)
print("Missing values before (sample):", missing_before[missing_before > 0].to_dict())
print("Missing values after (sample):", missing_after[missing_after > 0].to_dict())
print(f"Income missing before imputation: {income_missing_before}, filled with median: {income_median}")

Initial shape: (2240, 34)
Final shape: (2240, 34)
Duplicates before: 0
Duplicates after: 0
Missing values before (sample): {}
Missing values after (sample): {'dt_customer': 1324}
Income missing before imputation: 0, filled with median: 51381.5
